<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/Dense_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.5 MB/s eta 0:00:00


In [ ]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

api_key = 'vo8ANuN8dbBGoDDhe5H5cKrYxDjknz5PLgYs6Ba5'
co = cohere.Client(api_key)

In [ ]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

In [ ]:
texts = text.split('.')
texts = [t.strip() for t in texts]

In [ ]:
response = co.embed(
    texts = texts,
    input_type="search_document",
).embeddings
embeds = np.array(response)
print(embeds.shape)

(15, 4096)


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.5 MB/s eta 0:00:00


In [ ]:
import faiss
dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeds))

True


In [ ]:
def search(query, number_of_results=3):
  query_embed = co.embed(texts=[query], input_type="search_query",).embeddings[0]
  distances, similar_item_ids = index.search(np.float32([query_embed]), number_of_results)
  texts_np = np.array(texts)
  results = pd.DataFrame(data={'texts': texts_np[similar_item_ids[0]], 'distance':distances[0]})
  print(f"Query: '{query}'\nNearest neighbors:")
  return results

In [ ]:
query = "how precise was the science"
results = search(query)
results

Query: 'how precise was the science'
Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,10757.365234
1,Caltech theoretical physicist and 2017 Nobel l...,11566.132812
2,Interstellar uses extensive practical and mini...,11922.839844


In [ ]:
pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
  tokenize_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
      tokenize_doc.append(token)
  return tokenize_doc

tokenized_corpus = []
for passage in tqdm(texts):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

def keyword_search(query, top_k=3, num_candidates=15):
  print("input question:", query)
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
  print(f'Top-3 lexical search (BM25) hits')
  for hit in bm25_hits[0: top_k]:
    print('\t{:.3f}\t{}'.format(hit['score'], texts[hit['corpus_id']].replace('\n','')))



100%|██████████| 15/15 [00:00<00:00, 38456.33it/s]


In [ ]:
keyword_search(query='how precise was the science')

input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine


In [ ]:
query = "What is the mass of moon"
results = search(query)
results

Query: 'What is the mass of moon'
Nearest neighbors:


,texts,distance
0,Cinematographer Hoyte van Hoytema shot it on 3...,13381.806641
1,The film had a worldwide gross over $677 milli...,13683.921875
2,It has also received praise from many astronom...,13780.046875


In [ ]:
query = 'how precise was science'
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.14535783),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'), index=10, relevance_score=0.0946777),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight'), index=11, relevance_score=0.0079997545)]

In [ ]:
for idx, result in enumerate(results.results):
  print(idx, result.relevance_score, result.document.text)

0 0.14535783 It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
1 0.0946777 The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
2 0.0079997545 It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight


In [ ]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
  print('input question:', query)
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x:x['score'], reverse=True)

  print(f'Top-3 lexical search (BM25) hits')
  for hit in bm25_hits[0: top_k]:
    print('\t{:.3f}\t{}'.format(hit['score'], texts[hit['corpus_id']].replace('\n','')))

  docs = [texts[hit['corpus_id']] for hit in bm25_hits]
  print(f'\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)')
  results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
  for hit in results.results:
    print('\t{:.3f}\t{}'.format(hit.relevance_score, hit.document.text.replace('\n', ' ')))

In [ ]:
keyword_and_reranking_search(query = 'how precise was the science')

input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles

Top-3 hits by rank-API (10 BM25 hits re-ranked)
	0.170	It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
	0.070	The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
	0.004	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Scien

In [ ]:
query = 'income generated'


results = search(query)

docs_dict = [{'text': text} for text in results['texts']]
response = co.chat(
    message=query,
    documents=docs_dict,
)
print(response.text)

Query: 'income generated'
Nearest neighbors:
The film Interstellar generated over $677 million worldwide, or $773 million including subsequent re-releases.


In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-03-28 15:35:32--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1743179732&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzE3OTczMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4

In [ ]:
pip install --upgrade llama-cpp-python langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=6008069 sha256=cab8e49b6ef6a461f6c81cd9a6f1a14b9ba604c6b931fea49f09ecf9b3075c6e
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.47
    Uni

In [ ]:
from langchain import LlamaCpp

llm = LlamaCpp(
  model_path="Phi-3-mini-4k-instruct-fp16.gguf",
  n_gpu_layers=-1,
  max_tokens=500,
  n_ctx=2048,
  seed=42,
  verbose=False
)


llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
  model_name="thenlper/gte-small"
)

<ipython-input-35-490cb528fa26>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_texts(texts, embedding_model)

Query: 'income generated'
Nearest neighbors:
The film Interstellar generated a worldwide gross of over $677 million, and $773 million with subsequent re-releases, making it the tenth-highest grossing film of 2014.


In [ ]:
from langchain import PromptTemplate

template = """<|user|>
Relevat informaction
{context}

Provide a concise answer the following questin using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [ ]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db.as_retriever(),
  chain_type_kwargs={"prompt": prompt},
  verbose=True
)

In [42]:
rag.invoke("Income generated")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Income generated',
 'result': ' The film generated over $677 million worldwide in gross, with a total of approximately $773 million after subsequent re-releases.'}